In [15]:
def import_clean_data(file_path):
    # Combine the date and time columns to one
    df = pd.read_csv(file_path, parse_dates=[["Date", "Time"]])
    # Remove Columns that are not useful to us
    df.drop(["Time Zone", "Transaction ID", "Payment ID", "Device Name", "Details", "Location", "Dining Option",
             "Customer ID", "Customer Name", "Customer Reference ID"], axis=1, inplace=True)
    # Remove the Refunds
    df = df[df.Category != "None"]
    # Convert columns to appropriate types
    # Dollar columns to floats using regex to remove dollar sign
    dollar_column = df[df.columns[7:11]].replace('[$]', '', regex=True)
    # Update df with proper dollar columns
    df.update(dollar_column)
    # Change the dollar columns to float type
    df[["Gross Sales", "Discounts", "Net Sales", "Tax"]] = df[["Gross Sales", "Discounts", "Net Sales", "Tax"]]\
        .apply(pd.to_numeric)

    return df

In [16]:
import pandas as pd

# Options to make pd easier to read from print
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.width = 900

In [17]:
data = import_clean_data("Tea Room/2017-2018/item_sales.csv")
print(data.describe())
print(data.head(10))

                 Qty    Gross Sales      Discounts      Net Sales            Tax
count  103460.000000  103460.000000  103460.000000  103460.000000  103460.000000
mean        1.023671       3.395213      -0.086345       3.308868       0.327317
std         0.190461       2.715676       0.342503       2.754802       0.267416
min        -2.000000      -6.750000     -11.400000      -6.750000      -0.880000
25%         1.000000       2.150000       0.000000       2.000000       0.110000
50%         1.000000       2.750000       0.000000       2.650000       0.250000
75%         1.000000       3.880000       0.000000       3.850000       0.470000
max         8.000000     120.000000       2.150000     120.000000      10.480000
             Date_Time         Category         Item  Qty Price Point Name      SKU Modifiers Applied  Gross Sales  Discounts  Net Sales   Tax Notes Event Type
2  2018-04-13 17:36:17  Merchandise-MER  Jars of Tea  1.0            Large  005-RET               NaN        14

In [23]:
def demand(df):
    #TODO Only include coffee sales as those are the only orders the express line can handle
    
    # Group All of the rows by weekday, hour, and minute, returns a groupBy object which is a collection of data frames
    # weekday: 0 = monday, 1 = tuesday, ... 6 = sunday
    grouped = df.groupby([df["Date_Time"].dt.weekday.rename('Day_of_Week'), df["Date_Time"].dt.hour.rename('Hour'), 
                          df["Date_Time"].dt.minute.rename('Minute')])
    
    # Get the number of days in the data frame
    num_days = len(df["Date_Time"].dt.normalize().unique())
    
    # Get how many orders are placed on that day of the week, at that time, at that minute
    num_occurrences = grouped.size().to_frame('Number of Orders').reset_index()
    # Get Average Number of orders on that day and time
    num_occurrences['Average Number of Orders'] = num_occurrences['Number of Orders'].apply(lambda x: x/num_days)
    
    return num_occurrences

In [24]:
def overwhelmed(orders_per_hour):
    df = demand(data)
    
    # Find times where average number of orders is greater than what we can handle
    busy_times = df['Average Number of Orders'] > orders_per_hour
    
    print(df[busy_times])
    
    
overwhelmed(.5)

      Day_of_Week  Hour  Minute  Number of Orders  Average Number of Orders
0               0     7      39                 1                  0.007634
1               0     7      55                 3                  0.022901
2               0     7      56                 1                  0.007634
3               0     7      57                 2                  0.015267
4               0     7      58                 2                  0.015267
5               0     7      59                 1                  0.007634
6               0     8       0                 6                  0.045802
7               0     8       1                 5                  0.038168
8               0     8       2                 8                  0.061069
9               0     8       3                 8                  0.061069
10              0     8       4                11                  0.083969
11              0     8       5                15                  0.114504
12          

In [ ]:
## TODO Find a way to plot data
def graph_df(data):
    %matplotlib inline
    import matplotlib.pyplot as plt
    
    occurrences = demand(data)
    print(occurrences)

    #occurrences.plot(y='num_occurences', x=['Hour','Minute'], figsize=(20,10))


#graph_df(data)